# DS400: Assignment 5 (Python)

##### Tingyu Lin (Student ID: 5638679)


In [ ]:
import requests
import numpy as np
import pandas as pd
import time

In [ ]:
# hyperlink: https://github.com/ltyita/Python_Assign05.git